In [109]:
import os

import zlib
import hashlib




def git_init(path):
    try:
        os.mkdir(f'{path}/.git')
        os.mkdir(f'{path}/.git/objects')
        os.mkdir(f'{path}/.git/refs')
        with open(f'{path}/.git/HEAD', 'w') as head_file:
            head_file.write('ref: refs/heads/master\n')
        

        return f'Initialized empty Git repository in {path}/.git'

    except:
        return'Error: Git repository already exits'


def git_cat_file(key_hash):
    
    if len(key_hash) != 40:
            return 'Wrong key_hash'
    try:
        
        first_key = key_hash[:2]
        second_key = key_hash[2:]

        with open(f'{PATH_OBJ}/{first_key}/{second_key}', 'r') as f:
            compressed_data = eval(f.read())
            #print(compressed_data)
            decompressed_data = zlib.decompress(compressed_data)

            return decompressed_data

    except:
        return 'Error cat-file'


def made_key_hash(object_name, content):
    header = f"{object_name} {len(content)}\u0000"


    key_hash = hashlib.sha1(
        header.encode('utf-8')
    ).hexdigest()

    return key_hash


def git_hash_object(file_path):
    if os.path.exists(file_path):
        if os.path.isdir(file_path):
            object_name = 'tree'
        else:
            object_name = 'blob'
        original_data  = open(file_path, 'rb').read()
        compressed_data = zlib.compress(original_data, level=-1)
        #print(original_data, compressed_data)
        key_hash = made_key_hash(object_name, str(original_data))

        if not os.path.exists(f'{PATH_OBJ}/{key_hash[:2]}'):
            os.mkdir(f'{PATH_OBJ}/{key_hash[:2]}')

        with open(f'{PATH_OBJ}/{key_hash[:2]}/{key_hash[2:]}', 'w') as f:
            f.write(str(compressed_data))

        return key_hash

    else:
        return 'hash_object: file does not exist'
     



def git_ls_tree(key_hash):

    content = git_cat_file(key_hash)


    return content.decode('utf-8')



def git_write_tree(path):
    wr_tree = ''
    y = []
    for file_name in set(os.listdir(path)) - set(['.git']):
        #print(file_name)
        file_path = f'{path}/{file_name}'

        if os.path.isdir(file_path):
            object_name = 'tree'

            key = git_write_tree(file_path)
            #print(file_name)
        else:
            object_name = 'blob'

            key = git_hash_object(file_path)

        
        wr_tree += f'{object_name} {key} {file_name}\n'
    
    key =  made_key_hash('tree', path)

    if not os.path.exists(f'{PATH_OBJ}/{key[:2]}'):
        os.mkdir(f'{PATH_OBJ}/{key[:2]}')

    with open(f'{PATH_OBJ}/{key[:2]}/{key[2:]}', 'w') as f:
        f.write(str(zlib.compress(bytes(wr_tree, 'utf-8'), level=-1)))

    return key


PATH = input('cd ') 
PATH_GIT = PATH + '/.git'
PATH_OBJ = PATH_GIT + '/objects'

'C:/cs102/homework10/.git/objects'



cd C:/cs102/homework10


'C:/cs102/homework10/.git/objects'

In [110]:
git_init(PATH)


'Initialized empty Git repository in C:/cs102/homework10/.git'

In [111]:
hash_name = git_write_tree(PATH_GIT)
git_write_tree(PATH_GIT)

'edf11449702254288f0448ac8287b649517c0c80'

In [112]:
print(git_ls_tree(hash_name))

tree 4cb483684810d2e38e1fdc6ddcabd50c0dc5f0dc refs
blob b83577636fce3249e23013752c8073388ac38245 HEAD
tree 66a4c093b2091f7f3ab0b1b3b0f9b9b165bced7f objects



In [113]:
print(git_cat_file('b83577636fce3249e23013752c8073388ac38245'))

b'ref: refs/heads/master\r\n'
